# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [46]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [47]:
url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=84&sort=sale'

In [49]:
hdr = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

def my_get(url):
    return requests.get(url, headers=hdr)

#response = requests.get(url, headers=hdr)
response = my_get(url)

In [50]:
#response = requests.get(url)
results = response.json()
flattened_data = json_normalize(results)

flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1.head(2)

,brand_name,family_articles,flags,is_premium,media,name,outfits,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,url_key
0,Tommy Hilfiger,"[{'sku': 'TO122P03I-K11', 'url_key': 'tommy-hi...","[{'key': 'discountRate', 'value': '-29%', 'tra...",False,[{'path': 'TO/12/2P/03/IK/11/TO122P03I-K11@13....,SLIM FIT - Polo - sky captain,NaN,False,False,False,False,"69,95 €","50,00 €",clothing,"[S, M, L, XL]",TO122P03I-K11,tommy-hilfiger-polo-sky-captain-to122p03i-k11
1,YOURTURN,"[{'sku': 'YO112O03G-Q11', 'url_key': 'yourturn...","[{'key': 'discountRate', 'value': '-31%', 'tra...",False,[{'path': 'YO/11/2O/03/GQ/11/YO112O03G-Q11@11....,Baskets basses - black/red,NaN,False,False,False,False,"34,95 €","24,00 €",shoe,"[40, 41, 42, 43, 45]",YO112O03G-Q11,yourturn-baskets-basses-blackred-yo112o03g-q11


In [51]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [52]:
# Get the total number of pages

total_pages=results['pagination']['page_count']

# Your code
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    if i%100==0:
        print(i)
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset={k}&sort=sale'
    #response = requests.get(url)promo-enfant&limit=84&offset={k}&sort=sale
    response=my_get(url)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)
df.head(20)

0


C:\Users\eleouetd01\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


ConnectionError: HTTPSConnectionPool(host='www.zalando.fr', port=443): Max retries exceeded with url: /api/catalog/articles?categories=promo-homme&limit=84&offset=5208&sort=sale (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000024EB0C88160>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [ ]:
# Display the trending brand in DataFrame
df.brand_name.value_counts().index[0]

In [ ]:
df.columns

In [ ]:
df.drop(['url_key','price.has_different_original_prices','price.has_different_prices','price.has_different_promotional_prices','price.has_discount_on_selected_sizes_only','tracking_information.impression_beacon','tracking_information.metrigo_impression_urls','tracking_information.source','amount'],axis=1,inplace=True)
#droping the columns I don't need

In [ ]:
df.columns

In [ ]:
df.dtypes

2. The product(s) with the highest discount.

In [ ]:
#THIS IS AN EXAMPLE OF THE LECON. Display the brand with maximal total discount 

#Our data is still text. Convert prices into numbers:
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df_discount=df.copy()

total_disc=df_discount.groupby(['brand_name']).sum().discount_amount
total_disc.sort_values(ascending=False).index[0]
# Display the brand with maximal total discount

In [ ]:
# df1 is a copy of df but with new column discount amount
df_discount.head(1)

In [ ]:
df_discount['discount_amount'].max()
#highest price of discount amoung all goods

In [ ]:
df_discount['discount_amount'].idxmax()

In [ ]:
max_disc=df_discount.groupby('sku').max().discount_amount
max_disc.sort_values(ascending=False).index[0]

In [ ]:
max_disc.sort_values(ascending=False).head(1)
#Here is the product with highest discount

In [ ]:
df_discount.rename(columns={"price.promotional": "price_promo", "price.original": "price_orig"}).head(1)

3. The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [ ]:
df_discount['discount_amount'].sum()

In [ ]:
df_discount['price_promo']

In [ ]:
sum_original_prices=df['price.original'].sum()

In [ ]:
sum_original_prices